<a href="https://colab.research.google.com/github/daliduraku/Tesla-Stock-Monthly-Forecasting/blob/main/Tesla_Stock_Monthly_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tesla Stock Monthly Forecasting - by Dali Duraku

##Data Import and Cleaning


In [1]:
import pandas as pd
import plotly.express as px

data = pd.read_csv('Tesla Stock Price History.csv')
data

,Date,Price,Open,High,Low,Vol.,Change %
0,12/01/2024,462.28,352.55,488.31,348.22,1.60B,33.93%
1,11/01/2024,345.16,252.04,361.93,238.88,2.09B,38.15%
2,10/01/2024,249.85,262.67,273.54,212.11,1.91B,-4.50%
3,09/01/2024,261.63,215.26,264.86,209.64,1.61B,22.19%
4,08/01/2024,214.11,227.69,231.87,182.00,1.62B,-7.74%
...,...,...,...,...,...,...,...
115,05/01/2015,16.72,15.33,16.86,14.68,1.34B,10.95%
116,04/01/2015,15.07,12.58,15.92,12.40,1.48B,19.75%
117,03/01/2015,12.58,13.51,13.75,12.09,1.84B,-7.17%
118,02/01/2015,13.56,13.60,15.03,12.89,1.60B,-0.13%


In [2]:
data.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis=1, inplace=True)
data

,Date,Price
0,12/01/2024,462.28
1,11/01/2024,345.16
2,10/01/2024,249.85
3,09/01/2024,261.63
4,08/01/2024,214.11
...,...,...
115,05/01/2015,16.72
116,04/01/2015,15.07
117,03/01/2015,12.58
118,02/01/2015,13.56


In [3]:
fig = px.line(data, x='Date', y='Price', title='Tesla Monthly Stock Price over Time')
fig.show()

## Data Preparation for Training the Model (Rolling Window)

In [4]:
def rolling_window_df(data, window_size):
    # Generate a Dataframe with rolling windows of closing prices.
    columns = [f'N-{window_size - i}' for i in range(window_size)] + ['N']
    df = pd.DataFrame(columns=['Target Date'] + columns)

    for i in range(len(data) - window_size):
        row = [data.index[i + window_size]] + list(data[i:i + window_size + 1])
        df.loc[i] = row

    return df

window_size = 5
data_with_dates = data.set_index('Date')['Price']
window_df = rolling_window_df(data_with_dates, window_size)
window_df

,Target Date,N-5,N-4,N-3,N-2,N-1,N
0,07/01/2024,462.28,345.16,249.85,261.63,214.11,232.07
1,06/01/2024,345.16,249.85,261.63,214.11,232.07,197.88
2,05/01/2024,249.85,261.63,214.11,232.07,197.88,178.08
3,04/01/2024,261.63,214.11,232.07,197.88,178.08,183.28
4,03/01/2024,214.11,232.07,197.88,178.08,183.28,175.79
...,...,...,...,...,...,...,...
110,05/01/2015,13.80,16.56,16.60,17.74,17.88,16.72
111,04/01/2015,16.56,16.60,17.74,17.88,16.72,15.07
112,03/01/2015,16.60,17.74,17.88,16.72,15.07,12.58
113,02/01/2015,17.74,17.88,16.72,15.07,12.58,13.56


In [5]:
X = window_df.iloc[:, 1:-1].values
y = window_df['N'].values

X.shape, y.shape

((115, 5), (115,))

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((92, 5), (23, 5), (92,), (23,))

In [7]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

model.intercept_, model.coef_

(0.7609793405573839,
 array([-0.15014176,  0.31178811, -0.03636747, -0.12149503,  0.973423  ]))

## Visualization and Evaluation


In [8]:
# Predict on training set
y_train_pred = model.predict(X_train)

# Create a DataFrame for visualization
train_dates = window_df['Target Date'].iloc[:-len(y_test)].reset_index(drop=True)
train_results = pd.DataFrame({'Date': train_dates, 'Actual': y_train, 'Predicted': y_train_pred})

# Plot the predictions vs actual values
fig = px.line(train_results, x='Date', y=['Actual', 'Predicted'], title='Actual vs Predicted Stock Prices on Training Data (Monthly)')
fig.show()

In [9]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')

Mean Absolute Error: 1.29


In [10]:
# Visualize the predictions vs actual values
test_dates = data['Date'].iloc[-len(y_test):]
results = pd.DataFrame({'Date': test_dates, 'Actual': y_test, 'Predicted': y_pred})

fig = px.line(results, x='Date', y=['Actual', 'Predicted'], title='Actual vs Predicted Stock Prices')
fig.show()